In [17]:
#m Se importan las librerias necesarias
from datetime import datetime
import pandas as pd
import numpy as np
import datetime
import collections
import pandas_datareader.data as web

In [13]:
# Se importa la librería para importar los historiales de metatraderfrom datetime import datetime
import MetaTrader5 as mt5

In [14]:
local_exe = 'C:\Program Files\FxPro - MetaTrader 5\terminal64.exe'
# Fechas de descarga
fi = datetime.datetime(2022, 2, 14, 0, 0) #Fecha de inicio de historicos
ff = datetime.datetime(2022, 3, 22, 0, 0)# Fecha final 


La siguiente función será utilizada para poder extraer y leer los historicos de las operaciones de las distintas cuentas en Metratrade5-

In [18]:
def f_leer_archivo(numcue: int, contra: str, direc: str, fi, ff):
    usuario = mt5.f_init_login(numcue, contra, direc)
    usuinfo = mt5.f_acc_info(mt5_client)
    hist = mt5.f_hist_trades(param_ct=usuario, usuinfo=fi, hist=ff)#Precios historicos
    return hist